Setup data

In [367]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import iris
from ascend import shape


In [368]:
def box_plot(data, ax, edge_color, fill_color, positions, widths):
    bp = ax.boxplot(data, patch_artist=True, positions = positions, widths=widths, showfliers=False, whis=[10,90])
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        if element == 'medians':
            col = 'black'
        else:
            col = edge_color
        plt.setp(bp[element], color=col)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp


def bxp(data, ax, colour, alpha, position, width, **kwargs):
    bp = ax.bxp(data, patch_artist=True, positions=position, widths=width, showfliers=False, **kwargs)
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        if element == 'medians':
            col = 'black'
        else:
            col = colour
        plt.setp(bp[element], color=col)
        plt.setp(bp[element], alpha=alpha)

    for patch in bp['boxes']:
        patch.set(facecolor=colour)
        patch.set(alpha = alpha)   
        
    return bp

    
def create_x_points(ys, basex, offset):
    xs = []
    for i, v in enumerate(ys):
        if i == 0:
            xs.append(basex)
            vm1 = v
        else:
            if abs(v - vm1) <= 1:
                if xs[i-1] < basex:
                    xs.append(basex + offset)
                elif xs[i-1] == basex:
                    xs[i-1] = basex - offset
                    xs.append(basex + offset)
                else:
                    # previous version has been offset positively
                    xs.append(basex - offset)
            else:
                xs.append(basex)
            vm1 = v
    
    return xs


def mask_wp2_data(cube, shp):
    # mask wp2 atlas data using shape file
    
    # due to extra dimensions in the WP2 atlas data need to go about
    # it in a slightly roundabout way
    # first get lat / lon mask over 2 dimensions
    xy_mask = shp.cube_intersection_mask(cube[0,:,:,0])
    # then broadcast to correct shape
    xyp_mask = np.broadcast_to(xy_mask[:,:,np.newaxis], cube.shape[1:])
    # correct shape mask
    cube_mask = np.logical_not(np.broadcast_to(xyp_mask, cube.shape))
    # finally combine with existing mask
    cube_mask = np.logical_or(cube_mask, cube.data.mask)

    # apply to cube
    cube.data.mask = cube_mask

    return cube


def load_wp2_atlas(method, var):
    # load netCDF file
    base_path = "/net/home/h03/hadcy/python/lines_of_evidence/constrained_atlas"

    # load shape for masking
    mask_shp = shape.load_shp(
        '/net/home/h02/tcrocker/code/EUCP_WP5_Lines_of_Evidence/shape_files/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp',
        name='Romania'
    )[0]

    bxp_obs = []
    for type in ["cons", "uncons"]:
        fname = f"{base_path}/atlas_EUCP_{method}_{type}_{var}.nc"
        
        cube = iris.load_cube(fname)

        # mask the cube and select first time point (JJA)
        cube = mask_wp2_data(cube, mask_shp)[0]

        # area average
        cube.coord("latitude").units = "degrees"
        cube.coord("latitude").guess_bounds()
        cube.coord("longitude").units = "degrees"
        cube.coord("longitude").guess_bounds()
        grid_areas = iris.analysis.cartography.area_weights(cube)
        cube_mean = cube.collapsed(["latitude", "longitude"], iris.analysis.MEAN, weights=grid_areas)

        # create boxplot stats object
        if type == "cons":
            label = method
        else:
            label = None

        bxp_stats = {
            "whislo": cube_mean.extract(iris.Constraint(percentile=10)).data.item(),
            "q1": cube_mean.extract(iris.Constraint(percentile=25)).data.item(),
            "med": cube_mean.extract(iris.Constraint(percentile=50)).data.item(),
            "q3": cube_mean.extract(iris.Constraint(percentile=75)).data.item(),
            "whishi": cube_mean.extract(iris.Constraint(percentile=90)).data.item(),
            "label": label
        }

        bxp_obs.append(bxp_stats)

    return bxp_obs
    

In [369]:

# considering results from this paper:
# Niculiţă, M. (2020). Landslide Hazard Induced by Climate Changes in North-Eastern Romania. 
# Climate Change Management, (May), 245–265. https://doi.org/10.1007/978-3-030-37425-9_13

# global variables to control everything
RECIPE_RUN = 'recipe_GCM_and_RCM_Romania_20211118_151728'
BASE_PATH = f'/home/h02/tcrocker/code/EUCP_WP5_Lines_of_Evidence/esmvaltool/esmvaltool_output/{RECIPE_RUN}/work/boxplots/main/'
SEASON = 'JJA'
CLIMWIP_PATH = '/home/h02/tcrocker/code/EUCP_WP5_Lines_of_Evidence/weighting_data/ClimWIP/pr_weighted_CMIP5_Romania_1995-2014_2041-2060.nc'

INSTITUTES = [
    'IPSL',
    'NCC',
    'MPI-M',
    'CNRM-CERFACS',
    'ICHEC',
    'MOHC',
    'KNMI',
    'HCLIMcom',
    'SMHI'
]


def remove_institute_from_driver(driver_str):
    # remove the institute bit from the "driver" string
    new_str = driver_str
    # loop through the institutes and remove them if found
    for i in INSTITUTES:
        i = '^' + i + '-'
        new_str = re.sub(i, '', new_str)

    if new_str == driver_str:
        raise ValueError(f"No institute found to remove from {driver_str}")

    return new_str


# read data
cmip5 = pd.read_csv(f'{BASE_PATH}CMIP5_{SEASON}.txt', sep=':', header=None)
cmip6 = pd.read_csv(f'{BASE_PATH}CMIP6_{SEASON}.txt', sep=':', header=None)
cordex = pd.read_csv(f'{BASE_PATH}CORDEX_{SEASON}.txt', sep=':', header=None)
cpm = pd.read_csv(f'{BASE_PATH}CPM_{SEASON}.txt', sep=':', header=None)
ukcp_g = pd.read_csv(f'{BASE_PATH}UKCP_gcm_{SEASON}.txt', sep=':', header=None)
ukcp_r = pd.read_csv(f'{BASE_PATH}UKCP_rcm_{SEASON}.txt', sep=':', header=None)

niculita_model_list = [
    'RCA4 MPI-M-MPI-ESM-LR',
    'RCA4 MOHC-HadGEM2-ES',
    'RCA4 ICHEC-EC-EARTH',
    'RCA4 CNRM-CERFACS-CNRM-CM5',
    'REMO2009 MPI-M-MPI-ESM-LR',
    'RACMO22E MOHC-HadGEM2-ES',
    'RACMO22E ICHEC-EC-EARTH', 
    'HIRHAM5 ICHEC-EC-EARTH',
    ]

cpm_driver_list = [
    'ICTP-RegCM4-7-0 MOHC-HadGEM2-ES',
    'SMHI-HCLIM38-ALADIN ICHEC-EC-EARTH',
]


wp2_methods = {
    "ETHZ_CMIP6_ClimWIP": "CMIP6",
    "ICTP_CMIP6_REA": "CMIP6",
    "ICTP_CMIP5_REA": "CMIP5",
    "UKMO_CMIP6_UKCP": "UKCP_GCM"
}


colour_map = {
    "CMIP6": "tab:blue",
    "CMIP5": "tab:orange",
    "CORDEX": "tab:green",
    "CPM": "tab:red",
    "UKCP_GCM": "tab:purple",
}

# create subset of models used in paper
nic_df = cordex[cordex[0].isin(niculita_model_list)]
cpm_driver_df = cordex[cordex[0].isin(cpm_driver_list)]

# calculate CMIP5 CORDEX drivers from CORDEX model names
cordex_driver_list = list(
    set(
        [remove_institute_from_driver(n.split(' ')[1]) for n in cordex[0]]
    )
)
cordex_driver_df = cmip5[cmip5[0].isin(cordex_driver_list)]

# chuck everything in a dataframe for plotting
plot_df = pd.DataFrame(
    {
        "CMIP6": cmip6[1],
        "CMIP5": cmip5[1],
        "CORDEX Drivers": cordex_driver_df[1],
        "CORDEX": cordex[1],
        "Niclulită models": nic_df[1],
        "CPM Drivers": cpm_driver_df[1],
        "CPM": cpm[1],
        "UKCP_GCM": ukcp_g[1],
        "UKCP_RCM": ukcp_r[1],
        "UKCP Drivers": ukcp_g[ukcp_g[0].isin(ukcp_r[0])][1],
    }
)

# also need ClimWIP data
pr_cmip5_climwip = iris.load_cube(CLIMWIP_PATH)

# load WP2 constraint data
constraint_data = {}
for m in wp2_methods.keys():
    constraint_data[m] = load_wp2_atlas(m, "pr")

/net/project/ukmo/scitools/opt_scitools/environments/default/2021_03_18-1/lib/python3.6/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
/net/project/ukmo/scitools/opt_scitools/environments/default/2021_03_18-1/lib/python3.6/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
/net/project/ukmo/scitools/opt_scitools/environments/default/2021_03_18-1/lib/python3.6/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
/net/project/ukmo/scitools/opt_scitools/environments/default/2021_03_18-1/lib/python3.6/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
/net/project/ukmo/scitoo

In [370]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [371]:
plt.rcParams.update({'font.size': 14})
# create figure and axes
f, axs = plt.subplots(1,4, sharey=True, figsize=[19.2 ,  9.77], gridspec_kw={'width_ratios': [3, 2, 4, 1]})
f.suptitle("Projected % change in summer (JJA) rainfall for Romania. 2041-2060 vs 1995-2014. RCP8.5/ssp585")
# size of dots in swarm plots
swarm_size = 7

In [372]:
# First panel

# plot GCM boxes
axs[0].clear()
# matplotlib
box_plot([cmip6[1], cmip5[1], ukcp_g[1]], axs[0], "black", "None", [0, 1, 2], [0.5, 0.5, 0.5])
# box_plot([pr_cmip5_climwip[2].data], axs[0], "grey", "lightgrey", [1], [0.375])
# axs[0].boxplot(
#     x=[cmip6[1], cmip5[1], pr_cmip5_climwip[2].data],
#     positions=[0, 1, 1],
#     whis=[10,90],
#     showfliers=False,
#     widths=[0.5, 0.5, 0.375],
#     patch_artist=True
# )

# plot dots
sns.swarmplot(
    data=plot_df[["CMIP6", "CMIP5", "UKCP_GCM"]], ax=axs[0],
    size=swarm_size, palette=["tab:blue", "tab:orange", "tab:purple"],
    alpha=0.75
    )
# seaborn
#sns.boxplot(data=plot_df[["CMIP6", "CMIP5"]], ax=axs[0], color='white', fliersize=0)
axs[0].axhline(linestyle=":", color="k", alpha=0.5)
plt.setp(axs[0].get_xticklabels(), rotation=45, ha="right")

axs[0].set_title("GCMs")

Text(0.5, 1.0, 'GCMs')

In [373]:
# Now plot CORDEX related stuff
box_plot([cordex[1]], axs[1], "black", "None", [1], [0.5])
# axs[1].boxplot(x=cordex[1], whis=[10, 90], showfliers=False, positions=[1])
sns.swarmplot(
    data=plot_df[["CORDEX Drivers", "CORDEX", "Niclulită models"]],
    ax=axs[1], size=swarm_size, palette=["tab:orange", "tab:green", "tab:green"]
)
axs[1].axhline(linestyle=":", color="k", alpha=0.5)
plt.setp(axs[1].get_xticklabels(), rotation=45, ha="right")
# divider lines
axs[1].axvline(0.5, color="lightgrey")
axs[1].axvline(1.5, color="lightgrey")
axs[1].set_title("CORDEX")

Text(0.5, 1.0, 'CORDEX')

In [374]:
# plot constrained ranges. 2nd panel
axs[1].clear()
for i, k in enumerate(constraint_data.keys()):
    colour = colour_map[wp2_methods[k]]
    # constrained
    bxp([constraint_data[k][0]], axs[1], colour, 0.75, [i], 0.375)
    # unconstrained
    bxp([constraint_data[k][1]], axs[1], colour, 0.25, [i], 0.5)

axs[1].axhline(linestyle=":", color="k", alpha=0.5)
plt.setp(axs[1].get_xticklabels(), rotation=45, ha="right")
axs[1].set_title("Uncertainty estimates\nfrom GCMs and observations")

Text(0.5, 1.0, 'Uncertainty estimates\nfrom GCMs and observations')

In [375]:
constraint_data[k][0]

{'whislo': -54.41280748499521,
 'q1': -35.16062123862237,
 'med': -12.476393291797445,
 'q3': 3.8226843398839887,
 'whishi': 26.32323455402991,
 'label': 'UKMO_CMIP6_UKCP'}

In [376]:
# third panel
sns.swarmplot(
    data=plot_df[["CPM Drivers", "CPM"]], ax=axs[2],
    size=swarm_size, palette=["tab:green", "tab:red"]
)
axs[2].axhline(linestyle=":", color="k", alpha=0.5)
plt.setp(axs[2].get_xticklabels(), rotation=45, ha="right")
# divider lines
axs[2].axvline(0.5, color="lightgrey")
axs[2].set_title("CPMs")

Text(0.5, 1.0, 'CPMs')

In [377]:
# or third panel like Carol's
axs[2].clear()
sns.swarmplot(
    data=plot_df[["CMIP5", "CORDEX", "CPM", "UKCP_GCM", "UKCP_RCM"]], 
    ax=axs[2],
    size=swarm_size,
    palette=["tab:orange", "tab:green", "tab:red", "tab:purple", "tab:purple"]
)
# CORDEX drivers
y = plot_df["CORDEX Drivers"].dropna()
x = create_x_points(y, 0.5, 0.05)
axs[2].scatter(x, y, color="tab:orange", marker=">", s=50)

# CPM drivers
y = plot_df["CPM Drivers"].dropna()
x = create_x_points(y, 1.5, 0.05)
axs[2].scatter(x, y, color="tab:green", marker=">", s=50)

axs[2].axvline(2.5, color="lightgrey")

# UKCP drivers
y = plot_df["UKCP Drivers"].dropna()
x = create_x_points(y, 3.5, 0.05)
axs[2].scatter(x, y, color="tab:purple", marker=">", s=50)

axs[2].axhline(linestyle=":", color="k", alpha=0.5)
plt.setp(axs[2].get_xticklabels(), rotation=45, ha="right")
axs[2].set_title("Downscaled Projections")

Text(0.5, 1.0, 'Downscaled Projections')

In [378]:
# models from study
axs[3].clear()
sns.swarmplot(
    data = plot_df["Niclulită models"],
    ax=axs[3],
    size=swarm_size,
    palette=["tab:green"]
    )
axs[3].set_xticklabels(["Niclulită models"])
axs[3].axhline(linestyle=":", color="k", alpha=0.5)
plt.setp(axs[3].get_xticklabels(), rotation=45, ha="right")
axs[3].set_title("From study")


Text(0.5, 1.0, 'From study')

In [379]:
# set spacing etc.
axs[0].set_ylabel("%")
plt.tight_layout()
plt.subplots_adjust(bottom=0.18, wspace=0.06)

Text(183.72222222222223, 0.5, '%')